<a href="https://colab.research.google.com/github/AromaR/685_Project/blob/main/POMO_Probing_refactored_bert_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import torch
assert torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")

Found device: Tesla T4, n_gpu: 1


In [ ]:
import pandas as pd
import numpy as np
!pip install transformers
!pip install tqdm
from tqdm import tqdm

In [ ]:
# from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained("mrm8488/spanbert-large-finetuned-tacred")

# model = AutoModel.from_pretrained("mrm8488/spanbert-large-finetuned-tacred")

In [ ]:
train_data = []
with open("/content/drive/My Drive/POMO/emb_04_train_base", "r") as f:
  s = f.read()
  s = s.replace("grad_fn=<SelectBackward>","")
  s = s.replace(" ","")
  s = s.replace("(","")
  s = s.replace(")","")
  s = s.replace("[","")
  s = s.replace("]","")
  s = s.replace("tensor","")
  s = s.replace("\n","")
  s = s.replace("'","")
  h = s.split("|")
  for i in h:
    if i:
      y = i.split("$")#y[1] is 0/1
      x = y[0].split(",")
      temp = []
      for a in x:
        if a:
          c = a.split("e")
          if len(c)>1:
            c[1] = c[1].replace("0","",1)
            c[0] = c[0]+"e"+c[1]
          temp.append(float(c[0]))
      if len(y)>1:
        train_data.append([[[temp]],int(y[1])])



KeyboardInterrupt: ignored

In [ ]:
with open("/content/drive/My Drive/POMO/emb_05_train_base", "r") as f:
  s = f.read()
  s = s.replace("grad_fn=<SelectBackward>","")
  s = s.replace(" ","")
  s = s.replace("(","")
  s = s.replace(")","")
  s = s.replace("[","")
  s = s.replace("]","")
  s = s.replace("tensor","")
  s = s.replace("\n","")
  s = s.replace("'","")
  h = s.split("|")
  for i in h:
    if i:
      y = i.split("$")#y[1] is 0/1
      x = y[0].split(",")
      temp = []
      for a in x:
        if a:
          c = a.split("e")
          if len(c)>1:
            c[1] = c[1].replace("0","",1)
            c[0] = c[0]+"e"+c[1]
          temp.append(float(c[0]))
      if len(y)>1:
        train_data.append([[[temp]],int(y[1])])


In [ ]:
print(len(train_data))

In [ ]:
test_data = []
with open("/content/drive/My Drive/POMO/emb_04_test_base", "r") as f:
  s = f.read()
  s = s.replace("grad_fn=<SelectBackward>","")
  s = s.replace(" ","")
  s = s.replace("(","")
  s = s.replace(")","")
  s = s.replace("[","")
  s = s.replace("]","")
  s = s.replace("tensor","")
  s = s.replace("\n","")
  s = s.replace("'","")
  h = s.split("|")
  for i in h:
    if i:
      y = i.split("$")#y[1] is 0/1
      x = y[0].split(",")
      temp = []
      for a in x:
        if a:
          c = a.split("e")
          if len(c)>1:
            c[1] = c[1].replace("0","",1)
            c[0] = c[0]+"e"+c[1]
          temp.append(float(c[0]))
      if len(y)>1:
        test_data.append([[[temp]],int(y[1])])


In [ ]:
print(len(test_data))

In [ ]:
train = []
test = []
valid = []
trainans = []
testans = []
validans = []
for i in train_data:
  train.append(i[0])
  trainans.append(i[1])
for i in test_data:
  test.append(i[0])
  testans.append(i[1])
valid = train[15000:]
validans = trainans[15000:]
train = train[0:15000]
trainans = trainans[0:15000]


In [ ]:
# dataset = pd.read_csv("/content/drive/My Drive/POMO/train.pm",sep='\\t')
# dataarr = []
# ansarr = []
# for i in range(15000,20000):
#   dataarr.append(dataset.iloc[i,0])
#   ansarr.append(0)
#   dataarr.append(dataset.iloc[i,3])
#   ansarr.append(1)
# torch.set_printoptions(profile="full")


In [ ]:
# with open("/content/drive/My Drive/POMO/emb_04_train", "w") as writer:
#   for i in tqdm(range(0, len(dataarr), 128)):
#     batch = dataarr[i:i+128]
#     ans = ansarr[i:i+128]
#     inputs = tokenizer.batch_encode_plus(batch,
#                 add_special_tokens=True,
#                 truncation=True,
#                 padding=True,
#                 return_tensors="pt",
#             )
#     for j in range(0,128):
#       try:
#         writer.write("|")
#         writer.write(str(model(torch.tensor(inputs['input_ids'][j]).unsqueeze(0))[0][0][0]))
#         writer.write("$")
#         writer.write(str(ans[j]))
#       except:
#         continue
# torch.set_printoptions(profile="default")
    


In [ ]:
dataset = pd.read_csv("/content/drive/My Drive/POMO/test.pm",sep='\\t')
dataarr_test = []
# ansarr_test = []
# print("here we add to a 2d array, a sentence and a number 1/0 according to whether it contains appositions. The first sentence at pos 0 is not appositional, the one at position 3 is")
for i in range(0,len(dataset)):
  dataarr_test.append(dataset.iloc[i,0])
#   ansarr_test.append(0)
  dataarr_test.append(dataset.iloc[i,3])
#   ansarr_test.append(1)

In [ ]:
# dataset = pd.read_csv("/content/drive/My Drive/POMO/valid.pm",sep='\\t')
# dataarr_valid = []
# ansarr_valid = []
# print("here we add to a 2d array, a sentence and a number 1/0 according to whether it contains appositions. The first sentence at pos 0 is not appositional, the one at position 3 is")
# for i in range(0,len(dataset)):
#   dataarr_valid.append(dataset.iloc[i,0])
#   ansarr_valid.append(0)
#   dataarr_valid.append(dataset.iloc[i,3])
#   ansarr_valid.append(1)

In [ ]:
# print(len(dataarr))
# print(dataarr[0])
# print(ansarr[0])
# print(dataarr[1])
# print(ansarr[1])
# h = []
# for j in ansarr:
#   h.append(torch.LongTensor([j]).unsqueeze(0))
# ansarr1 = torch.stack((h))
# print(ansarr1.shape)

In [ ]:
# print(len(dataarr_test))
# print(dataarr_test[0])
# print(ansarr_test[0])
# print(dataarr_test[1])
# print(ansarr_test[1])
# ansarr1_test = torch.from_numpy(np.array(ansarr_test))

In [ ]:
# print(len(dataarr_valid))
# print(dataarr_valid[0])
# print(ansarr_valid[0])
# print(dataarr_valid[1])
# print(ansarr_valid[1])
# ansarr1_valid = torch.from_numpy(np.array(ansarr_valid))

Creation of data

Conversion to hugging faces BERT-Tacred fine-tuned representation

In [ ]:
class POMO_Dataset(torch.utils.data.dataset.Dataset):
    def __init__(self, _dataset):
        self.dataset = _dataset

    def __getitem__(self, index):
        example= self.dataset[0][index]
        target = self.dataset[1][index]
        target.squeeze(0)
        return np.array(example), target

    def __len__(self):
        return len(self.dataset)

In [ ]:
# emb_data = tokenizer.batch_encode_plus(dataarr_valid)
# k = [torch.LongTensor(x) for x in emb_data['input_ids']]
# max_len = max([len(x) for x in emb_data['input_ids']])
# data = [torch.nn.functional.pad(x, pad=(0, max_len - x.numel()), mode='constant', value=0) for x in k]
# data = torch.stack(data)
# valid = []
# for i in range(0,len(data)):
#   valid.append([data[i],ansarr1_valid[i]])


In [ ]:
# emb_data = tokenizer.batch_encode_plus(dataarr)
# k = [torch.LongTensor(x) for x in emb_data['input_ids']]
# max_len = max([len(x) for x in emb_data['input_ids']])
# data = [torch.nn.functional.pad(x, pad=(0, max_len - x.numel()), mode='constant', value=0) for x in k]
# data = torch.stack(data)
# train = []



Get Bert embeddings

In [ ]:
# input_ids = torch.tensor(tokenizer.encode("Hello, my name is Aroma")).unsqueeze(0)  # Batch size 1
# outputs = model(input_ids)
# last_hidden_states = outputs[0]

In [ ]:
# print(len(last_hidden_states[0][0])) #CLS baby!

In [ ]:
# ansarr.unsqueeze(0).unsqueeze(0)

In [ ]:
# a = []
# for i in data:
#   a.append(i.unsqueeze(0).unsqueeze(0))

In [ ]:
# a = torch.stack(a)

In [ ]:
# print(a.shape)

In [ ]:
# train = []
# train = [a,ansarr1]

In [ ]:
# print(data)

In [ ]:
# print((train[0][1]))

In [ ]:

# test_loader = torch.utils.data.DataLoader(dataset=POMO_Dataset(test),
#                                            batch_size=4,
#                                            shuffle=True)
# valid_loader = torch.utils.data.DataLoader(dataset=POMO_Dataset(valid),
#                                            batch_size=4,
#                                            shuffle=True)
# train_loader = torch.utils.data.DataLoader(dataset=POMO_Dataset(train),
#                                            batch_size=4,
#                                            shuffle=True)


  
  


In [ ]:
# for batch_images, targets in train_loader:
#     print(batch_images.shape)
#     print(targets)
#     break

In [ ]:
metrics = [] #we will add mean accuracy on task, control task 1 and control task 2 here



Probing task level 1 : Classifier - binary, Aroma's CNN

In [ ]:
# import torch.nn as nn
# import torch.nn.functional as F


# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 2, 1,)
#         self.fc1 = nn.Linear(801, 120)
#         self.fc2 = nn.Linear(120, 1)

#     def forward(self, x):
#         x = (F.relu(self.conv1(x.long()))).long()
#         x = F.relu(self.fc1(x)).long()
#         x = (self.fc2(x)).long()
#         return x


# net = Net()
    
   

In [ ]:
# import torch.optim as optim

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [ ]:
# for epoch in range(2):  # loop over the dataset multiple times

#     running_loss = 0.0
#     for i, data in enumerate(train_loader, 0):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = net(inputs.long())
#         print(labels)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             running_loss = 0.0

# print('Finished Training')

In [ ]:
!pip install keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
model = Sequential()
model.add(Conv2D(64, kernel_size=1, activation='relu', input_shape=(1,1,768)))
model.add(Conv2D(32, kernel_size=1, activation='relu'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
train.extend(valid)
trainans.extend(validans)

In [ ]:
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))

Control Task: Let us see if our model is too complex and makes sense of nonsense

In [ ]:
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))

This is a prediction rate that is about the same as a random rate. Let us see if the information learned by the model is by virtue of the language embeddings or not

In [ ]:
cont_train = []
for i in train:
  cont_train.append([[np.add(np.array(i[0][0]),np.random.uniform(low=-1, high=1, size=len(i[0][0]) )).tolist()]])
cont_test = []
for i in test:
  cont_test.append([[np.add(np.array(i[0][0]),np.random.uniform(low=-1, high=1, size=len(i[0][0]) )).tolist()]])

In [ ]:
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))

In [ ]:
metrics.append((a,b,c))

As we can see the randomized addition to the embeddings has resulted in worse accuracy. Let us now experiment with a different version of the model

In [ ]:
from keras.models import Sequential

from keras.layers import Dense, Conv2D, Flatten

model = Sequential()

model.add(Conv2D(128, kernel_size=1, activation='relu', input_shape=(1,1,768)))

model.add(Conv2D(64, kernel_size=1, activation='relu'))
model.add(Conv2D(32, kernel_size=1, activation='relu'))

model.add(Flatten())

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c))




In [ ]:
a,b,c,d,e,f = [],[],[],[],[],[]
for i in train:
  a.append([i[0][0]])
for j in test:
  b.append([j[0][0]])
for l in cont_train:
  d.append([l[0][0]])
for n in cont_test:
  f.append([n[0][0]])


In [ ]:
print(len(a[0][0]))

In [ ]:
train = a
test = b
cont_train = d
cont_test = f

In [ ]:
from keras.models import Sequential

from keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D, Embedding

# Define the Keras model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu',input_shape=(1,768)))
model.add(Dropout(0.50))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dropout(0.50))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c))


In [ ]:
from keras.models import Sequential

from keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D, Embedding

# Define the Keras model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu',input_shape=(1,768)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c))

In [ ]:
from keras.models import Sequential

from keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D, Embedding

# Define the Keras model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu',input_shape=(1,768)))
model.add(Conv1D(filters=16,kernel_size=2, padding='same', activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c))

In [ ]:
!pip install keras-tuner


In [ ]:
import kerastuner as kt
import keras
!pip install tensorflow
import tensorflow as tf

In [ ]:
from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c))

In [ ]:
from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c))


In [ ]:

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0001)
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c))

In [ ]:

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.01)
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c))

In [ ]:
from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)
hist = model.fit(train, trainans, validation_data=(test, testans), epochs=30)
preds = model.predict(test)
a = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(train, np.random.choice([0, 1], size=len(trainans)).tolist(), validation_data=(test, np.random.choice([0, 1], size=len(testans)).tolist()), epochs=30)
b = (np.mean(hist.history.get("val_accuracy")))
hist = model.fit(cont_train, trainans, validation_data=(cont_test, testans), epochs=30)
c = (np.mean(hist.history.get("val_accuracy")))
metrics.append((a,b,c))
almostright = []
totallywrong = []
m = 0
for i,j in zip(preds,testans):
  if (i>0.4 and i<0.5 and j==1) or (i<0.6 and i>0.5 and j==0):
    almostright.append((m,i))
  if (i<0.20 and j==1) or (i>0.80 and j==0):
    totallywrong.append((m,i))
  m = m+1

In [ ]:
for i in almostright[0:10]:
  print(dataarr_test[i[0]],i[1])


In [ ]:
for i in totallywrong[0:10]:
  print(dataarr_test[i[0]],i[1])